In [1]:
import datetime
import cv2
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [2]:
start = datetime.datetime.now()
sift = cv2.xfeatures2d.SIFT_create()
image_path_list = []

for label in os.listdir("BOW"):
    image_path_list.append(os.path.join("BOW", label))
    
des_list = []
des_list1 = []
labels = []
count = 0
for path in image_path_list:
    for file_name in os.listdir(path):
        #print(file_name)
        count = count +1
        img = cv2.imread(os.path.join(path,file_name),2)
        #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        kp,des = sift.detectAndCompute(img,None)
        des_list1.append(des)
        labels.append(path)
        for i in range(len(des)):
            if i%5 == 0: 
                des_list.append(des[i])
#des_list = np.asarray(des_list)
#des_list = np.concatenate(des_list,axis=0)
count
end = datetime.datetime.now()
print(end-start)

0:00:52.527763


In [5]:

import pickle
with open('test2.pkl', 'wb') as handle:
    pickle.dump((labels), handle)

In [4]:
start = datetime.datetime.now()
kmeans = KMeans(n_clusters=250).fit(des_list)
end = datetime.datetime.now()
print(end-start)

0:28:27.265338


In [5]:
visual_word = kmeans.cluster_centers_

In [6]:
def build_histogram(descriptor_list, cluster_alg):
    histogram = np.zeros(len(cluster_alg.cluster_centers_))
    cluster_result =  cluster_alg.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return histogram

In [9]:
preprocessed_image = []
label = []
for path in image_path_list:
    for file_name in os.listdir(path):
        img = cv2.imread(os.path.join(path,file_name))
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        kp,des = sift.detectAndCompute(gray,None)
        if (des is not None):
            histogram = build_histogram(des, kmeans)
            preprocessed_image.append(histogram)
            label.append(path)

In [10]:
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(preprocessed_image,label, test_size=0.3, random_state=0)

clf = neighbors.KNeighborsClassifier(n_neighbors = 10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy1 %.2f %%" %(100*accuracy_score(y_test, y_pred)))


Accuracy1 59.76 %


In [11]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train,y_train)
y_predict = clf.predict(X_test)
print("Accuracy2 %.2f %%" %(100*accuracy_score(y_test, y_predict)))

Accuracy2 66.57 %


In [12]:
import pickle
with open('test1.pkl', 'wb') as handle:
    pickle.dump((kmeans,visual_word,preprocessed_image), handle)